In [ ]:

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import random
from sklearn.metrics import confusion_matrix
import itertools



In [ ]:

train_data = []
test_data = []

for root, _, files in os.walk('/kaggle/input/yoga-poses-dataset'):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            label = os.path.basename(root)  # Assuming folder name is the label

            # Adjust this part based on where you want to add the data
            if 'TRAIN' in root:  # Check if 'TRAIN' is in the folder path
                train_data.append([image_path, label])
            elif 'TEST' in root:  # Check if 'TEST' is in the folder path
                test_data.append([image_path, label])


train_df = pd.DataFrame(train_data, columns=['image', 'label'])
test_df = pd.DataFrame(test_data, columns=['image', 'label'])

print(test_df.head())



In [ ]:
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


class MyDataset(Dataset):
    def __init__(self, df, image_column, label_column, transform=None):
        self.df = df
        self.image_column = image_column
        self.label_column = label_column
        self.transform = transform

        # Create and fit a LabelEncoder for labels
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(self.df[self.label_column])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_path = self.df.iloc[idx][self.image_column]

        # Load the image using PIL (Pillow)
        image = Image.open(image_path).convert('RGB')

        label = self.df.iloc[idx][self.label_column]

        # Transform label to numerical using LabelEncoder
        label = self.label_encoder.transform([label])[0]

        # Apply transformations only if they are provided
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# Define the target size for your images
target_size = (224, 224)  # Example: Resize to 224x224

# Modified transformations with resizing
train_transform = transforms.Compose([
    transforms.Resize(target_size),  # Resize the image
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize(target_size),  # Resize the image
    transforms.ToTensor(),
])

In [ ]:
def preprocess_images(df, image_column, transform):
    """Resizes and transforms images in a DataFrame."""
    for index in df.index:
        image_path = df.loc[index, image_column]
        try:
            # Load the image
            image = Image.open(image_path).convert('RGB')

            # Print image size before transformation (for debugging)
            print(f"Original image size: {image.size}")

            # Apply the transformation
            transformed_image = transform(image)

            # Print image size after transformation (for debugging)
            print(f"Transformed image size: {transformed_image.shape}")

            # Save the transformed image back to the same path
            os.remove(image_path)  # Remove original image before saving
            transformed_image_pil = transforms.ToPILImage()(transformed_image)
            transformed_image_pil.save(image_path)

        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            # Handle the error (e.g., skip the image, remove it from the DataFrame)


In [ ]:
def custom_collate(batch):
    """Pads images in a batch to the maximum size."""
    # Get maximum image dimensions
    max_width = max(image.shape[2] for image, label in batch)
    max_height = max(image.shape[1] for image, label in batch)

    # Pad images
    padded_images = []
    labels = []
    for image, label in batch:
        pad_width = max_width - image.shape[2]
        pad_height = max_height - image.shape[1]
        padded_image = torch.nn.functional.pad(image, (0, pad_width, 0, pad_height))
        padded_images.append(padded_image)
        labels.append(label)

    # Stack padded images and labels
    return torch.stack(padded_images), torch.tensor(labels)

batch_size = 16

In [ ]:
train_dataset = MyDataset(train_df, image_column='image', label_column='label', transform=train_transform)
test_dataset = MyDataset(test_df, image_column='image', label_column='label', transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size,collate_fn=custom_collate, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,collate_fn=custom_collate, shuffle=False)

In [ ]:
# Define the model (similar to the TensorFlow model)
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),
            nn.Flatten(),
            nn.Linear(256 * 14 * 14, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Instantiate the model, loss function, and optimizer
num_classes = len(train_df['label'].unique())
model = CNNModel(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
### Training loop
epochs = 50
train_losses, train_accuracies = [], []
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

In [ ]:

# Plotting training loss and accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_accuracies, label='Train Accuracy')
plt.xlabel('Epoch')

# Plotting training loss and accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_accuracies, label='Train Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Train Loss')

# Produits payants Colab - Résilier les contrats ici
# En attente de finalisation de l'exécution actuelle.

plt.ylabel('Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()# l'exécution actuelle

# Evaluation on test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

In [ ]:
Model = CNNModel(num_classes=10)
torch.save(Model.state_dict(),'full_Model.pth')